<a href="https://colab.research.google.com/github/yigitcanolmez/Langchain-Exercises/blob/main/Build_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install python-dotenv

In [10]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) #read local .env

os.getenv("OPEN_AI_KEY")

In [11]:
#burada olmayan variable içinde tanım gerçekleştirilebiliyor sıfırdan yeni bir field açmışız gibi düşünülebilir.
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ""

In [ ]:


os.getenv("LANGCHAIN_API_KEY")


In [13]:
pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.7/336.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [14]:
import getpass
#burada direkt olarak kullanıcıdan bir değer alınarak variable set etme işlemi söz konusu

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# tokeni set ettikten sonra;

from langchain_openai  import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")


··········


In [15]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Guweyn Bravo")])

AIMessage(content='Hello, Guweyn Bravo! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 14, 'total_tokens': 28}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_c9aa9c0491', 'finish_reason': 'stop', 'logprobs': None}, id='run-88224adf-09f6-44a3-8adc-030eea80816a-0', usage_metadata={'input_tokens': 14, 'output_tokens': 14, 'total_tokens': 28})

In [16]:
model.invoke([HumanMessage(content="What's my name?")])
# model önceki verileri almadığı için ismi bilmiyor chat completions tarzı bir şey esasında direkt olarak. o
#o yüzden bir sonraki adıma gidiyoruz

AIMessage(content="I'm sorry, but I don't have access to personal information about users. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 11, 'total_tokens': 32}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'stop', 'logprobs': None}, id='run-1b143994-5805-42dd-b89b-f346b6e041bb-0', usage_metadata={'input_tokens': 11, 'output_tokens': 21, 'total_tokens': 32})

In [17]:
from langchain_core.messages import AIMessage

response = model.invoke(
    [
        HumanMessage(content="Hi! I'm Guweyn Bravo"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

response.content

'You mentioned that your name is Guweyn Bravo. How can I assist you today, Guweyn?'

Şimdi sırada bir array olarak metinleri tutmaktan ziyade direkt olarak bir data store içerisinde mesaj geçmişini tutmak var. O yüzden langchain_community paketini indirmemiz gerekiyor.

In [18]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.7 MB/s eta 0:00:00


In [19]:
!pip install langchain_core

In [20]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)

from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)



In [21]:
config = {"configurable": {"session_id": "abc2"}}

In [22]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Hello, Bob! How can I assist you today?'

In [23]:
config = {"configurable": {"session_id": "abc3"}}

In [24]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. As of now, you haven't shared your name. How can I assist you today?"

In [25]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob. How can I help you today, Bob?'

Şimdi sırada prompt templates


In [26]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model



In [27]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Hi Bob! How can I assist you today?'

In [28]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [29]:
config = {"configurable": {"session_id": "abc5"}}

In [31]:
response = with_message_history.invoke(
    [HumanMessage(content="what was my name")],
    config=config,
)

response.content

'You mentioned that your name is Jim. How can I help you today, Jim?'

şimdi biraz daha karışık işler yapmak adına system prompt içerisine değişken ataması gerçekleştirelim

In [32]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [34]:
response = chain.invoke(
    {"messages": [HumanMessage(content="Benim adım neydi")], "language": "Türkçe"}
)

response.content

'Üzgünüm, ama daha önce isminizi belirtmediğiniz için adınızı bilmiyorum. Size nasıl yardımcı olabilirim?'

In [40]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [41]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'¡Hola, Todd! ¿En qué puedo ayudarte hoy?'

model sınırlama


In [65]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="Im bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)
# burada amaç mesajları trim ederek 65 tokene kadar sondan başlayarak hafızaya alıyor. system ile beraber include system


[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [67]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's did i ask you before?")],
        "language": "English",
    }
)
response.content



'You asked, "What\'s 2 + 2?" and then you said "thanks."'

In [68]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked, "What\'s 2 + 2?"'

history tanımı yapalım

In [69]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [70]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="Hi I am Bob?")],
        "language": "English",
    },
    config=config,
)

response.content

'Hello, Bob! How can I assist you today?'

In [71]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what is my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'You mentioned that your name is Bob. How can I assist you further?'

In [72]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content)


Hi
 Todd
!
 Sure
,
 here's
 a
 joke
 for
 you
:


Why
 don't
 scientists
 trust
 atoms
?


Because
 they
 make
 up
 everything
!

